<a href="https://colab.research.google.com/github/jegork/marketing-analytics-kaggle/blob/main/Section1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>